https://www.tensorflow.org/tutorials/mnist/beginners/

download and read in the data automatically

In [2]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


MNIST被分成三份，mnist.train 55000,mnist.test 10000,mnist.validation 5000
然后每一部分又包含images和labels，mnist.train.images,mnist.train.labels

In [21]:
print(type(mnist.train.images))
print(mnist.train.images.shape)
# 55000-by-784, 每个图像是28-by-28,flatten之后是784
# mnist.train.images 就是一个tensor，tensor中元素（像素）值的取值范围是[0,1]
print("max : %f ; min: %f " % (mnist.train.images.max(),mnist.train.images.min()))

# labels 使用one-hot可以选择one-hot编码
print(mnist.train.labels[0])
#print(mnist.train.images[0])

<class 'numpy.ndarray'>
(55000, 784)
max : 1.000000 ; min: 0.000000 
[ 0.  0.  0.  0.  0.  0.  0.  1.  0.  0.]


numpy等是用c,fortran等实现，用于高效的数学和矩阵运算。使用tensorflow，仍然无法避免要使用python语言进行一些操作，这些操作一般会有较大的开销（overhead），尤其是GPU运算或者分布式计算的时候，数据传输花费时间。
tensorflow做了繁重的工作来避免计算过程中python操作带来的开销。并不是运行一个独立于python的操作，而是让我们描述一个可以交互操作的图。（一些其它的机器学习库也是这样做的，theano、torch）

python的作用是构建外部的图，决定图的那一部分应该运行。

In [19]:
import tensorflow as tf

构造图

In [30]:
#使用符号操作来描述图的交互
x = tf.placeholder(tf.float32, [None, 784])
#x 不是一个具体的值，而是 placeholder ,当run的时候才输入一个值。

#对于参数，我们需要weights和biases等，不用像上边x那样作为input，tf提供了Variable来处理这些参数
#Variable是可修改的tensor，存在于图的操作中，可以使用、计算、修改
W = tf.Variable(tf.zeros([784, 10]))
b = tf.Variable(tf.zeros([10])) # create and initialized it

y = tf.nn.softmax(tf.matmul(x, W) + b)
# y Tensor("Softmax_1:0", shape=(?, 10), dtype=float32)

定义loss

In [34]:
# 计算交叉熵
y_ = tf.placeholder(tf.float32, [None, 10])
# y_用于输入正确的类别标签

# y是[N, 10] y_也是[N, 10], elmentwise取log，然后elementwise计算乘积
# 这也是为什么将y用one-hot编码的原因，不希望被类别数值影响计算
#cross_entropy = tf.reduce_mean(-tf.reduce_sum(y_ * tf.log(y), reduction_indices = [1]))
#reduction_indices = [1]指定求和运算是在第二个维度计算的，将第二个维度（也就是横向）上的元素求和

#tutorial说上述那样求softmax的cross entropy会有计算不稳定的问题，不太了解
softmax_cross_entropy = tf.nn.softmax_cross_entropy_with_logits(labels = y_, logits = y)

In [35]:
learning_rate = 0.5
# 使用Gradient descent算法来minimize交叉熵，tf还提供很多其他的optimizer
train_step = tf.train.GradientDescentOptimizer(learning_rate).minimize(softmax_cross_entropy)

tf会在图中加入新的操作，实现[backpropagation](http://colah.github.io/posts/2015-08-Backprop/)和gradient descent

run

In [48]:
# 在运行之前还要create一个operation，初始化我们创建的所有Variables
# 这里只是定义，并没有运行
init = tf.global_variables_initializer()

#然后我们可以在一个Session中launch我们的模型
sess = tf.Session()
sess.run(init)

training_steps = 10000
batch_size = 100

for i in range(training_steps):
    batch_xs, batch_ys = mnist.train.next_batch(batch_size)
    sess.run(train_step, feed_dict = {x: batch_xs, y_: batch_ys})
#运行前边定义init操作和train_step操作

Evaluate

In [49]:
correct_predictions = tf.equal(tf.argmax(y, 1), tf.argmax(y_, 1))
accuracy = tf.reduce_mean(tf.cast(correct_predictions, tf.float32)) # accuracy也是一个operation

#然后来查看准确率
print(sess.run(accuracy, feed_dict={x:mnist.test.images, y_: mnist.test.labels}))

0.6663
